In [1]:
!pip install chromadb
!pip install langchain chromadb transformers sentence-transformers
!pip install langchain chromadb pymupdf
!pip install -U langchain-community
!pip install pymupdf

In [2]:
import fitz
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json
import torch

2025-05-07 05:02:07.624735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746594127.653259     481 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746594127.661550     481 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_name = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/2"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

Device set to use cuda:0


In [6]:
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_481/2691217058.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [7]:
def load_book_text(path='/kaggle/input/math-book/V.A.Zorich-Kniga-I-10-izdanie-Corr.pdf'):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [8]:
def split_text(text, max_len=2000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_len
        if end < len(text):
            last_newline = text.rfind('\n', start, end)
            if last_newline != -1 and last_newline > start:
                end = last_newline
        chunks.append(text[start:end].strip())
        start = end
    return chunks

In [9]:
def find_relevant_chunks(chunks, query, top_k=3):
    query_words = set(query.lower().split())
    scored_chunks = []
    for chunk in chunks:
        chunk_words = set(chunk.lower().split())
        score = len(query_words.intersection(chunk_words))
        scored_chunks.append((score, chunk))
    scored_chunks.sort(reverse=True, key=lambda x: x[0])
    return [chunk for score, chunk in scored_chunks[:top_k] if score > 0]

In [10]:
system_prompt_template = """
Ты - эксперт по математическому анализу. Используй следующий контекст из учебника для ответа на вопрос:

{context}

Вопрос студента:
{user_question}

Дай подробный и понятный ответ на русском языке.
"""

In [11]:
prompt = PromptTemplate(
    input_variables=["context", "user_question"],
    template=system_prompt_template
)

In [12]:
recommendation_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False
)

/tmp/ipykernel_481/2585610608.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  recommendation_chain = LLMChain(


In [13]:
def main():
    print("\nДобро пожаловать в систему подготовки к коллоквиуму по математическому анализу!")
    print("Задайте свой вопрос, и я постараюсь помочь, опираясь на учебный материал.\n")
    
    print("Загружаем и обрабатываем книгу, пожалуйста, подождите...")
    book_text = load_book_text('/kaggle/input/math-book/V.A.Zorich-Kniga-I-10-izdanie-Corr.pdf')
    
    chunks = split_text(book_text, max_len=2000)
    print(f"Книга разбита на {len(chunks)} частей.")
    
    while True:
        try:
            user_question = input("\nВведите ваш вопрос по математическому анализу (или 'выход' для завершения): ").strip()
            if user_question.lower() == 'выход':
                print("\nСпасибо за использование сервиса. Удачи на коллоквиуме!")
                break
            
            relevant_chunks = find_relevant_chunks(chunks, user_question, top_k=3)
            if not relevant_chunks:
                context_for_model = "К сожалению, в книге не найдено релевантного материала по вашему вопросу."
            else:
                context_for_model = "\n\n".join(relevant_chunks)
            
            answer = recommendation_chain.invoke({
                "context": context_for_model,
                "user_question": user_question
            })
            
            print("\n" + "="*40)
            print("ОТВЕТ".center(40))
            print("="*40)
            print(answer["text"].strip())
            
        except KeyboardInterrupt:
            print("\nРабота программы завершена пользователем.")
            break
        except Exception as e:
            print(f"\nПроизошла ошибка: {str(e)}. Попробуйте еще раз.")

In [ ]:
if __name__ == "__main__":
    torch.cuda.empty_cache()
    main()


Добро пожаловать в систему подготовки к коллоквиуму по математическому анализу!
Задайте свой вопрос, и я постараюсь помочь, опираясь на учебный материал.

Загружаем и обрабатываем книгу, пожалуйста, подождите...
Книга разбита на 572 частей.



Введите ваш вопрос по математическому анализу (или 'выход' для завершения):  Что такое предел функции?



                 ОТВЕТ                  
Ты - эксперт по математическому анализу. Используй следующий контекст из учебника для ответа на вопрос:

d) для любого множества X
card X < 2card X , в частности, n < 2n при любом n ∈N.
Указание. См. теорему Кантора в п. 1 § 4, гл. I.
. Пусть X1, …, Xm — конечная система конечных множеств. Покажите, что
card
 mS
i=1
Xi

=
P
i1
card Xi1 −
P
i1<i2
card(Xi1 ∩Xi2)+
P
i1<i2<i3
card(Xi1 ∩Xi2 ∩Xi3)−
−…+(−1)m−1 card(X1 ∩…∩Xm),
причем суммирование ведется по всевозможным наборам индексов в пределах
1, …, m, удовлетворяющих указанным под знаком суммы неравенствам.
. На отрезке [0, 1] ⊂R изобразите множество чисел x ∈[0, 1], троичная запись
которых x =0,α1α2α3… , где αi ∈{0, 1, 2}, обладает свойством:
a) α1 ̸=1;
b) (α1 ̸=1)∧(α2 ̸=1);
c) ∀i ∈N (αi ̸=1) (канторово множество).
. (Продолжение задачи 7.) Покажите, что
a) множество тех чисел x ∈[0, 1], троичная запись которых не содержит 1, равно-
мощно множеству всех чисел, двоичное представление которых 


Введите ваш вопрос по математическому анализу (или 'выход' для завершения):  Непрерывность функции в точке



                 ОТВЕТ                  
Ты - эксперт по математическому анализу. Используй следующий контекст из учебника для ответа на вопрос:

1. Непрерывность функции в точке (138). 2. Точки разрыва (142).
§ 2. Свойства непрерывных функций . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
145
1. Локальные свойства (145). 2. Глобальные свойства непрерывных
функций (147). Задачи и упражнения (155).
Глава V. Дифференциальное исчисление

§ 1. Дифференцируемая функция . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
160
1. Задача и наводящие соображения (160). 2. Функция, дифференци-
руемая в точке (165). 3. Касательная; геометрический смысл произ-
водной и дифференциала (167). 4. Роль системы координат (170). 5.
Некоторые примеры (172). Задачи и упражнения (177).
§ 2. Основные правила дифференцирования. . . . . . . . . . . . . . . . . . . . . . . . . . .
178
1. Дифференцирование и арифметические операции (178). 2. Диффе-
ренцирование ком


Введите ваш вопрос по математическому анализу (или 'выход' для завершения):  таблица производных



                 ОТВЕТ                  
Ты - эксперт по математическому анализу. Используй следующий контекст из учебника для ответа на вопрос:

1. Непрерывность функции в точке (138). 2. Точки разрыва (142).
§ 2. Свойства непрерывных функций . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
145
1. Локальные свойства (145). 2. Глобальные свойства непрерывных
функций (147). Задачи и упражнения (155).
Глава V. Дифференциальное исчисление

§ 1. Дифференцируемая функция . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
160
1. Задача и наводящие соображения (160). 2. Функция, дифференци-
руемая в точке (165). 3. Касательная; геометрический смысл произ-
водной и дифференциала (167). 4. Роль системы координат (170). 5.
Некоторые примеры (172). Задачи и упражнения (177).
§ 2. Основные правила дифференцирования. . . . . . . . . . . . . . . . . . . . . . . . . . .
178
1. Дифференцирование и арифметические операции (178). 2. Диффе-
ренцирование ком


Введите ваш вопрос по математическому анализу (или 'выход' для завершения):  Определение интеграла
